In [32]:
import clang.cindex

In [36]:

def find_typerefs(node, typename, prefix=""):
    print "{}{}  {} {}: type={} \t{} {}".format(prefix, node.location.line, node.spelling, node.kind, node.type.kind, node.spelling, node.displayname) 
    definition = clang.cindex.conf.lib.clang_getCursorDefinition(node);
    if definition:
        print "DEFINITION", definition.location.line, definition.kind, definition.spelling
        print "ARGS", clang.cindex.conf.lib.clang_Cursor_getTemplateArgumentType(node,0).spelling
        #     for x in node.get_children():
#         print x
    # Recurse for children of this node
    for c in node.get_children():
        find_typerefs(c, typename, (prefix + "->"))

In [34]:
code = """

/*
namespace epicure {

template <class T>
class RPC {T t;};

template <class T, typename ...Args>
auto publish(Args && ...args) {
    return RPC<T>();
}
}
*/

template <class T> 
int foo()
{
    return 10;
}

class Dog {

};

int main()
{
    //int cow;
  //  epicure::RPC<Dog> x = epicure::publish<Dog>();
    int bar = foo<Dog>();
    //int man;
}
"""

with open("code.cpp", "w") as text_file:
    text_file.write(code)

In [37]:
index = clang.cindex.Index.create()
tu = index.parse("code.cpp")
find_typerefs(tu.cursor, "Dog")

0  code.cpp CursorKind.TRANSLATION_UNIT: type=TypeKind.INVALID 	code.cpp code.cpp
->17  foo CursorKind.FUNCTION_TEMPLATE: type=TypeKind.FUNCTIONPROTO 	foo foo()
DEFINITION 17 CursorKind.FUNCTION_TEMPLATE foo
ARGS 
->->16  T CursorKind.TEMPLATE_TYPE_PARAMETER: type=TypeKind.UNEXPOSED 	T T
DEFINITION 16 CursorKind.TEMPLATE_TYPE_PARAMETER T
ARGS 
->->18   CursorKind.COMPOUND_STMT: type=TypeKind.INVALID 	 
->->->19   CursorKind.RETURN_STMT: type=TypeKind.INVALID 	 
->->->->19   CursorKind.INTEGER_LITERAL: type=TypeKind.INT 	 
->22  Dog CursorKind.CLASS_DECL: type=TypeKind.RECORD 	Dog Dog
DEFINITION 22 CursorKind.CLASS_DECL Dog
ARGS 
->26  main CursorKind.FUNCTION_DECL: type=TypeKind.FUNCTIONPROTO 	main main()
DEFINITION 26 CursorKind.FUNCTION_DECL main
ARGS 
->->27   CursorKind.COMPOUND_STMT: type=TypeKind.INVALID 	 
->->->30   CursorKind.DECL_STMT: type=TypeKind.INVALID 	 
->->->->30  bar CursorKind.VAR_DECL: type=TypeKind.INT 	bar bar
DEFINITION 30 CursorKind.VAR_DECL bar
ARGS 
->->->->-

In [87]:
x = tu.cursor.type.get_named_type().spelling

In [72]:
clang.cindex.conf.lib.

''

In [1]:
import clang.cindex
from clang.cindex import *
import cymbal
from ctypes import *

cymbal.monkeypatch_type('get_template_argument_type',
                        'clang_Type_getTemplateArgumentAsType',
                        [Type, c_uint],
                        Type)

cymbal.monkeypatch_type('get_num_template_arguments',
                        'clang_Type_getNumTemplateArguments',
                        [Type],
                        c_int)

# check if the cursor's type is a template
def is_template(node):
    return hasattr(node, 'type') and node.type.get_num_template_arguments() != -1



In [126]:
source = """
#include <string>
#include <iostream>

template <typename T> class X {};
X<bool> x;

template <class T>
int foo(int y) {return 10;}

template <class A, class B>
int baz(int y) {return 10;}

template <class T>
void publish() {;}

class Dog {
public:
    int bark(double volume, const std::string& language);
    void run(float speed) const { std::cout << "running"; }
};
class Cat {};

int main() {
    int x = foo<Dog>(7);
    int y = baz<Dog,Cat>(7);
    publish<Cat>();
    publish<Dog>();
    
}
"""

In [97]:
index = clang.cindex.Index.create()

# parse the TU
tu = clang.cindex.TranslationUnit.from_source('t.cpp', ['-std=c++11'], unsaved_files=[('t.cpp', source)])
for x in tu.diagnostics:
    print x

for c in tu.cursor.walk_preorder():
    if True: # is_template(c):
        t = c.type
        print "A", c.displayname, c.spelling, c.kind, t.kind, t.spelling, t.get_num_template_arguments()
        print "B", t.get_template_argument_type(0).spelling
        sp = c.semantic_parent
        lp = c.lexical_parent
        print "D", sp.displayname if sp else "None", lp.displayname if lp else "None"
        for x in c.get_arguments():
            print "C", x.type.spelling

A t.cpp t.cpp CursorKind.TRANSLATION_UNIT TypeKind.INVALID  -1
B 
D None None
A X<T> X CursorKind.CLASS_TEMPLATE TypeKind.INVALID  -1
B 
D t.cpp t.cpp
A T T CursorKind.TEMPLATE_TYPE_PARAMETER TypeKind.UNEXPOSED T -1
B 
D X<T> X<T>
A x x CursorKind.VAR_DECL TypeKind.UNEXPOSED X<bool> 1
B bool
D t.cpp t.cpp
A X X CursorKind.TEMPLATE_REF TypeKind.INVALID  -1
B 
D None None
A X X CursorKind.CALL_EXPR TypeKind.UNEXPOSED X<bool> 1
B bool
D x None
A foo(int) foo CursorKind.FUNCTION_TEMPLATE TypeKind.FUNCTIONPROTO int (int) -1
B 
D t.cpp t.cpp
A T T CursorKind.TEMPLATE_TYPE_PARAMETER TypeKind.UNEXPOSED T -1
B 
D foo(int) foo(int)
A y y CursorKind.PARM_DECL TypeKind.INT int -1
B 
D foo(int) foo(int)
A   CursorKind.COMPOUND_STMT TypeKind.INVALID  -1
B 
D foo(int) None
A   CursorKind.RETURN_STMT TypeKind.INVALID  -1
B 
D None None
A   CursorKind.INTEGER_LITERAL TypeKind.INT int -1
B 
D None None
A baz(int) baz CursorKind.FUNCTION_TEMPLATE TypeKind.FUNCTIONPROTO int (int) -1
B 
D t.cpp t.cpp
A A A

In [48]:
import clang.cindex
from clang.cindex import *
import cymbal
from ctypes import *

cymbal.monkeypatch_type('get_template_argument_type',
                        'clang_Type_getTemplateArgumentAsType',
                        [Type, c_uint],
                        Type)

cymbal.monkeypatch_type('get_num_template_arguments',
                        'clang_Type_getNumTemplateArguments',
                        [Type],
                        c_int)                
        

IndentationError: expected an indented block (<ipython-input-48-c19fb5735722>, line 19)

In [61]:
def find_all_published_types(cursor):
    types = []
    for node in cursor.walk_preorder():
        if node.type.kind == TypeKind.FUNCTIONPROTO and node.spelling == "publish":
            for child in node.get_children():
                if child.type.kind == TypeKind.RECORD:
                    types.append(child)
                    break
    return types

In [93]:
def find_definition(cursor, target_type_spelling):
    for node in cursor.walk_preorder():
        if node.kind == CursorKind.CLASS_DECL and node.type.spelling == target_type_spelling:
                return node

In [128]:
def describe_methods(definition_cursor):
    methods = {}
    # TODO: check for class or struct definition
    for node in definition_cursor.walk_preorder():
        if node.kind == CursorKind.CXX_METHOD:
            methods[node.spelling] = {
                "args": [{"name": arg.spelling, "type": arg.type.spelling} for arg in node.get_arguments()],
                "return": node.result_type.spelling,
            }
    return methods

In [129]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# parse the TU
tu = clang.cindex.TranslationUnit.from_source('t.cpp', ['-std=c++11'], unsaved_files=[('t.cpp', source)])
for x in tu.diagnostics:
    print x

types = find_all_published_types(tu.cursor)
for t in types:
    print t.location.line, t.spelling

for t in types:
    d = find_definition(tu.cursor, t.type.spelling)
    print d.location.line, d.type.spelling
    m = describe_methods(d)
    pp.pprint(m)

27 class Cat
28 class Dog
22 Cat
{   }
17 Dog
{   'bark': {   'args': [   {   'name': 'volume', 'type': 'double'},
                            {   'name': 'language',
                                'type': 'const std::string &'}],
                'return': 'int'},
    'run': {   'args': [{   'name': 'speed', 'type': 'float'}], 'return': 'void'}}


AttributeError: 'dict' object has no attribute 'dog'

In [ ]:
clang.cindex.Cursor.r